# Recognizing Flowers (Sunflower, Rose, Daisy, Tulip, Dandelion)

In [10]:
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.layers import Input
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils

## Loading Images

In [2]:
train_dir = './train'
test_dir = './test'

Adding data augmentation to better generalize the model

In [23]:
datagen = ImageDataGenerator(rescale=1./255, 
                             zoom_range=0.02, 
                             rotation_range = 0.02)
batch_size = 20

In [24]:
train_generator = datagen.flow_from_directory(train_dir,
                                            target_size=(150, 150), # all images will be resized to 150x150
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 750 images belonging to 5 classes.


In [25]:
test_generator = datagen.flow_from_directory(test_dir,
                                            target_size=(150, 150), # all images will be resized to 150x150
                                            batch_size=batch_size,
                                            class_mode='categorical')

Found 250 images belonging to 5 classes.


In [26]:
num_classes = 5
input_shape = (150, 150, 3)

### Approach 1: CNN

In [44]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras import optimizers
import keras
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs', 
                                          histogram_freq=0, 
                                          batch_size=32, 
                                          write_graph=True, 
                                          write_grads=False, 
                                          write_images=True, 
                                          embeddings_freq=0, 
                                          embeddings_layer_names=None, 
                                          embeddings_metadata=None)

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
# model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

model.add(Conv2D(64,(3,3), padding='same',activation='relu'))
# model.add(Dropout(0.2))

model.add(Conv2D(64,(3,3), padding='same',activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last")) 

model.add(Conv2D(128,(3,3), padding='same',activation='relu'))
model.add(Dropout(0.2))

model.add(Conv2D(128,(3,3), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), data_format="channels_last"))

model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_125 (Conv2D)          (None, 150, 150, 32)      896       
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 75, 75, 64)        18496     
_________________________________________________________________
conv2d_128 (Conv2D)          (None, 75, 75, 64)        36928     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 37, 37, 128)       73856     
__________

In [45]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.fit_generator(train_generator,
        epochs=20,
        validation_data=test_generator, 
        callbacks=[tb_callback])

Epoch 1/20
38/38 [==============================] - 113s 3s/step - loss: 1.6547 - acc: 0.2118 - val_loss: 1.6116 - val_acc: 0.2000
Epoch 2/20
38/38 [==============================] - 113s 3s/step - loss: 1.6044 - acc: 0.2328 - val_loss: 1.5783 - val_acc: 0.2440
Epoch 3/20
38/38 [==============================] - 115s 3s/step - loss: 1.4745 - acc: 0.3592 - val_loss: 1.4176 - val_acc: 0.3760
Epoch 4/20
38/38 [==============================] - 114s 3s/step - loss: 1.3548 - acc: 0.4474 - val_loss: 1.3213 - val_acc: 0.4400
Epoch 5/20
38/38 [==============================] - 114s 3s/step - loss: 1.2613 - acc: 0.4671 - val_loss: 1.4209 - val_acc: 0.4600
Epoch 6/20
38/38 [==============================] - 116s 3s/step - loss: 1.1530 - acc: 0.4974 - val_loss: 1.3860 - val_acc: 0.5320
Epoch 7/20
38/38 [==============================] - 113s 3s/step - loss: 1.1012 - acc: 0.5513 - val_loss: 1.2078 - val_acc: 0.4840
Epoch 8/20
38/38 [==============================] - 115s 3s/step - loss: 0.9503 - a

1. Data Augmentation (horizontal flip, rotation, zoom range) <br/>
0.5760
2. Remove horizontal flip <br/>
0.5520
3. Add an extra Dense layer with 512 nodes + extra dropout layer <br/>

### Approach 2: Using Pre-Trained Model - Inception V3 then add custom final Dense layers (Transfer Learning)

In [35]:
input_tensor = Input(shape=(150, 150, 3)) 
inception_base = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [41]:
num_train = 750
num_test = 250
train_features = np.zeros(shape=(num_train, 3, 3, 2048))
train_labels = np.zeros(shape=(num_train,5))